In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import sys
import scipy.stats as stats


In [3]:
counts=pd.read_json('reddit-counts.json.gz',lines=True)

In [4]:
#counts

In [5]:
counts['year'] = pd.DatetimeIndex(counts['date']).year
#counts

In [6]:
counts =(counts[counts['subreddit']=='canada'])

In [7]:
counts = counts[counts['year'] >= 2012]
counts = counts[counts['year'] <= 2013]
#counts

In [8]:
counts['dayofweek'] = counts['date'].dt.weekday
counts

,date,subreddit,comment_count,year,dayofweek
66,2013-03-14,canada,1657,2013,3
69,2013-07-08,canada,1369,2013,0
97,2012-07-04,canada,1343,2012,2
115,2013-03-11,canada,1619,2013,0
165,2013-09-11,canada,1909,2013,2
...,...,...,...,...,...
15389,2013-01-01,canada,2113,2013,1
15413,2013-07-27,canada,1070,2013,5
15430,2012-10-19,canada,1486,2012,4
15456,2012-01-15,canada,1256,2012,6


In [9]:
weekends=counts[(counts['dayofweek']).isin([5,6])]


In [10]:
weekends = weekends.reset_index(drop=True)

In [11]:
#weekends

In [12]:
weekdays=counts[~((counts['dayofweek']).isin([5,6]))]

In [13]:
weekdays = weekdays.reset_index(drop=True)

In [14]:
#weekdays

In [15]:
#Tests without transformation 
ttest=stats.ttest_ind(weekdays['comment_count'], weekends['comment_count'] ).pvalue
levenetest=stats.levene(weekdays['comment_count'], weekends['comment_count']).pvalue
weekdaysnormal=stats.normaltest(weekdays['comment_count']).pvalue
weekendsnormal=stats.normaltest(weekends['comment_count']).pvalue

In [16]:
#Transformations

In [17]:
#Logarithmic
weekdaysnormal_log = stats.normaltest(np.log(weekdays['comment_count'])).pvalue
weekendsnormal_log= stats.normaltest(np.log(weekends['comment_count'])).pvalue
levenetest_log = stats.levene(np.log(weekdays['comment_count']),np.log(weekends['comment_count'])).pvalue
levenetest_log

0.0004190759393372205

In [18]:
#square root
weekdaysnormal_sqrt = stats.normaltest(np.sqrt(weekdays['comment_count'])).pvalue
weekendsnormal_sqrt= stats.normaltest(np.sqrt(weekends['comment_count'])).pvalue
levenetest_sqrt = stats.levene(np.sqrt(weekdays['comment_count']),np.sqrt(weekends['comment_count'])).pvalue
levenetest_sqrt

0.5560544297516696

In [19]:
#square
levenetest_square=stats.levene(weekdays['comment_count']**2, weekends['comment_count']**2).pvalue
weekdaysnormal_square=stats.normaltest(weekdays['comment_count']**2).pvalue
weekendsnormal_square=stats.normaltest(weekends['comment_count']**2).pvalue
levenetest_square

7.391434727467261e-08

In [20]:
#Square root transformation is closest to the normal distributions so we pick that one
weekends=weekends.drop('year', axis=1)
weekdays=weekdays.drop('year', axis=1)


In [21]:
weekends

,date,subreddit,comment_count,dayofweek
0,2012-02-04,canada,1196,5
1,2012-11-17,canada,1570,5
2,2013-07-14,canada,908,6
3,2013-06-22,canada,984,5
4,2012-07-29,canada,1199,6
...,...,...,...,...
204,2012-11-04,canada,1772,6
205,2012-12-09,canada,1350,6
206,2012-12-02,canada,1725,6
207,2013-07-27,canada,1070,5


In [30]:
weekdays

,date,subreddit,comment_count,dayofweek,week
0,2013-03-14,canada,1657,3,11
1,2013-07-08,canada,1369,0,28
2,2012-07-04,canada,1343,2,27
3,2013-03-11,canada,1619,0,11
4,2013-09-11,canada,1909,2,37
...,...,...,...,...,...
517,2013-04-10,canada,2021,2,15
518,2013-03-19,canada,1630,1,12
519,2013-01-01,canada,2113,1,1
520,2012-10-19,canada,1486,4,42


In [23]:

weekdays['week'] = weekdays['date'].apply(lambda x: str(x.isocalendar()[1])).apply(pd.Series)
weekdays['year']= weekdays['date'].apply(lambda x: str(x.isocalendar()[0])).apply(pd.Series)
weekends['week'] = weekends['date'].apply(lambda x: str(x.isocalendar()[1])).apply(pd.Series)
weekends['year']= weekends['date'].apply(lambda x: str(x.isocalendar()[0])).apply(pd.Series)

In [26]:
weekdays_mean = weekdays.groupby(['year', 'week']).aggregate('mean').reset_index()
weekends_mean = weekends.groupby(['year', 'week']).aggregate('mean').reset_index()

In [27]:
weekly_weekday_normality_p= stats.normaltest(weekdays_mean['comment_count']).pvalue
weekly_weekend_normality_p= stats.normaltest(weekends_mean['comment_count']).pvalue
weekly_levene_p = stats.levene(weekdays_mean['comment_count'],weekends_mean['comment_count']).pvalue
weekly_ttest_p = stats.ttest_ind(weekdays_mean['comment_count'], weekends_mean['comment_count']).pvalue


In [28]:
utest_p= stats.mannwhitneyu(weekdays['comment_count'],weekends['comment_count'], alternative= 'two-sided').pvalue

In [29]:
weekly_ttest_p

1.3353656052303141e-34

In [30]:
weekly_levene_p

0.20383788083573426